In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import random

from src import cleaning

#### Read known sources to duplicate the same dataframe format

In [2]:
raw_data_path = '../data/raw/'

name = 'hd'
reviews_raw = pd.read_csv(raw_data_path + 'collected_reviews_' + name + '.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')
display(resumme_raw)
display(reviews_raw.sample(5))

,stars,reviews
0,5,2290
1,4,1308
2,3,396
3,2,132
4,1,128


,author,local_guide_info,rating,review,date_text,text_backup
279,Isabel Rodríguez Cedillo,Local Guide · 42 reseñas,5 estrellas,¡Buena comida!,Hace 5 años,Isabel Rodríguez Cedillo\nLocal Guide · 42 res...
2,Rainer Alfonso Ravelo,2 reseñas · 2 fotos,5 estrellas,Estuve cenando en familia el martes y además d...,Hace una semana,Rainer Alfonso Ravelo\n2 reseñas · 2 fotos\n\...
252,Loes van Mierlo,Local Guide · 52 reseñas · 96 fotos,4 estrellas,Un bonito y pequeño lugar para tomar una copa ...,Hace 8 años,Loes van Mierlo\nLocal Guide · 52 reseñas · 96...
339,Elvira fernandez,Local Guide · 12 reseñas · 1 foto,5 estrellas,NaN,Hace 9 meses,Elvira fernandez\nLocal Guide · 12 reseñas · 1...
223,Lena Font,6 reseñas,5 estrellas,"¡Muy buen lugar! El servicio es increíble, par...",Hace 2 años,Lena Font\n6 reseñas\n\n\n\n\n\n\nHace 2...


In [3]:
processed_data_path = '../data/processed/'
reviews_ready = pd.read_csv(processed_data_path + name + '_reviews.csv')
display(reviews_ready.head(3))

,review_id,review,local_guide_reviews,rating_score,service,meal_type,price_per_person_category,food_score,service_score,atmosphere_score,recommendations_list,date,avg_price_per_person
0,0,Además de que da un vibra de un bar de los 50’...,92.0,5.0,NaN,NaN,NaN,5.0,5.0,5.0,[''],2024-09-09,NaN
1,1,La hamburguesa de pollo frito estaba muy rica ...,35.0,4.0,Comí allí,Comida,30-40 €,4.0,5.0,4.0,[''],2024-08-01,40.0
2,2,Estuve cenando en familia el martes y además d...,2.0,5.0,NaN,NaN,NaN,5.0,5.0,5.0,[''],2024-09-16,NaN


#### Check target business

In [5]:
yelp_reviews_file = 'yelp_academic_dataset_review.json'
yelp_business_file = 'yelp_academic_dataset_business.json'

yelp_business = pd.read_json(os.path.join(raw_data_path, yelp_business_file), lines=True)
display(yelp_business.sample(3))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
17031,Gi5eYl4O8gJnfEJQyRSx7w,Alfa Romeo Of Wilmington Pike,99 Wilmington Pike,Chadds Ford,PA,19317,39.844149,-75.536905,3.5,5,1,None,"Used Car Dealers, Car Dealers, Automotive","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."
2057,ugSVRAKO6mxvsrElFg8USw,México Lindo,700 Moore St,Philadelphia,PA,19148,39.926402,-75.157537,4.5,73,1,"{'RestaurantsDelivery': 'True', 'HasTV': 'True...","Mexican, Restaurants","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
115375,QxGS9RR39IERlNw-KchKpw,Rouget,2 Swamp Rd,Newtown,PA,18940,40.229461,-74.940874,4.5,11,0,"{'GoodForKids': 'False', 'BusinessParking': '{...","French, Restaurants",None


In [6]:
yelp_business = yelp_business.sort_values(by='review_count', ascending=False).reset_index(drop=True)
display(yelp_business.head(20))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,_ab50qdWOk0DdB6XOrBitw,Acme Oyster House,724 Iberville St,New Orleans,LA,70130,29.954273,-90.068965,4.0,7568,1,"{'RestaurantsTakeOut': 'True', 'Alcohol': ''fu...","Live/Raw Food, Seafood, Restaurants, Cajun/Creole","{'Monday': '11:0-22:0', 'Thursday': '11:0-22:0..."
1,ac1AeYqs8Z4_e2X5M3if2A,Oceana Grill,739 Conti St,New Orleans,LA,70130,29.956231,-90.067563,4.0,7400,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Seafood, Cajun/Creole, Breakfast ...","{'Monday': '8:0-1:0', 'Tuesday': '8:0-1:0', 'W..."
2,GXFMD0Z4jEVZBCsbPf4CTQ,Hattie B’s Hot Chicken - Nashville,112 19th Ave S,Nashville,TN,37203,36.151387,-86.796603,4.5,6093,1,"{'RestaurantsGoodForGroups': 'True', 'Business...","American (Traditional), Chicken Shop, Southern...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-16:0', ..."
3,ytynqOUb3hjKeJfRj5Tshw,Reading Terminal Market,51 N 12th St,Philadelphia,PA,19107,39.953341,-75.158855,4.5,5721,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Candy Stores, Shopping, Department Stores, Fas...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
4,oBNrLz4EDhiscSlbOl8uAw,Ruby Slipper - New Orleans,200 Magazine St,New Orleans,LA,70130,29.951025,-90.067394,4.5,5193,1,"{'NoiseLevel': ''loud'', 'Caters': 'False', 'B...","Restaurants, American (Traditional), American ...","{'Monday': '0:0-0:0', 'Tuesday': '7:30-14:0', ..."
5,iSRTaT9WngzB8JJ2YKJUig,Mother's Restaurant,401 Poydras St,New Orleans,LA,70130,29.949057,-90.067049,3.5,5185,1,"{'RestaurantsGoodForGroups': 'True', 'BYOBCork...","Cajun/Creole, Restaurants, Event Planning & Se...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-17:0', '..."
6,VQcCL9PiNL_wkGf-uF3fjg,Royal House,441 Royal St,New Orleans,LA,70130,29.956473,-90.066386,4.0,5070,1,"{'WiFi': 'u'free'', 'Caters': 'False', 'Outdoo...","American (New), Restaurants, Sandwiches, Seafo...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
7,_C7QiQQc47AOEv4PE3Kong,Commander's Palace,1403 Washington Ave,New Orleans,LA,70130,29.928735,-90.084196,4.5,4876,1,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","French, Restaurants, Cocktail Bars, Nightlife,...","{'Monday': '0:0-0:0', 'Tuesday': '18:0-21:30',..."
8,GBTPC53ZrG1ZBY3DT8Mbcw,Luke,333 Saint Charles Ave,New Orleans,LA,70130,29.950742,-90.070416,4.0,4554,1,"{'RestaurantsReservations': 'True', 'Restauran...","German, Restaurants, Seafood, Cocktail Bars, F...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ..."
9,6a4gLLFSgr-Q6CZXDLzBGQ,Cochon,930 Tchoupitoulas St,New Orleans,LA,70130,29.942280,-90.067268,4.0,4421,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Cajun/Creole, Seafood, Restaurants","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21..."


#### Read reviews by chunks

In [7]:
# Initialize an empty list to store chunks
chunks = []
chunksize = 10000  # Adjust chunk size based on memory capacity

# Process reviews in chunks
for chunk in pd.read_json(os.path.join(raw_data_path, yelp_reviews_file), lines=True, chunksize=chunksize):
    merged_chunk = pd.merge(chunk, yelp_business, on='business_id', how='inner')
    chunks.append(merged_chunk)

# Concatenate all chunks into a single DataFrame
yelp_reviews = pd.concat(chunks, ignore_index=True)
yelp_reviews = yelp_reviews[['business_id', 'name', 'stars_x', 'date', 'text']]
yelp_reviews = yelp_reviews.rename(columns={'business_id':'place_id', 'name':'place', 'stars_x':'rating_score', 'text':'review'})
display(yelp_reviews.head(5))

,place_id,place,rating_score,date,review
0,XQfwVwDr-v0ZS3_CbbE5Xw,Turning Point of North Wales,3,2018-07-07 22:09:11,"If you decide to eat here, just be aware it is..."
1,7ATYjTIgM3jUlt4UM3IypQ,Body Cycle Spinning Studio,5,2012-01-03 15:28:18,I've taken a lot of spin classes over the year...
2,YjUWPpI6HXG530lwP-fb2A,Kettle Restaurant,3,2014-02-05 20:30:30,Family diner. Had the buffet. Eclectic assortm...
3,kxX2SOes4o-D3ZQBkiMRfA,Zaika,5,2015-01-04 00:01:03,"Wow! Yummy, different, delicious. Our favo..."
4,e4Vwtrqf-wpJfwesgvdgxQ,Melt,4,2017-01-14 20:54:15,Cute interior and owner (?) gave us tour of up...


#### Select place

In [23]:
reviews = yelp_reviews[yelp_reviews['place_id'] == 'ac1AeYqs8Z4_e2X5M3if2A']
place = reviews.iloc[0].place
reviews.drop(columns=['place_id', 'place'], inplace = True)
reviews.reset_index(drop = True, inplace = True)

print(place)
display(reviews)
print(reviews.shape)

Oceana Grill


/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_32742/766118533.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.drop(columns=['place_id', 'place'], inplace = True)


,rating_score,date,review
0,5,2016-08-04 00:07:32,If you are looking for great food and atmosphe...
1,2,2018-02-23 22:53:25,Disappointing experience. ITEMS ON MENU DO NOT...
2,5,2011-03-20 03:35:06,"Not much of a seafood lover, so I played it sa..."
3,5,2018-10-04 16:49:45,Really enjoyed the food and our waitress was s...
4,4,2018-06-19 13:38:11,This was our first place to eat in NOLA. The i...
...,...,...,...
7511,3,2021-12-22 12:41:52,The service at Oceana is very good. \nThe food...
7512,3,2018-11-04 20:35:44,Oceana's Famous Oysters\nFreshly shucked louis...
7513,4,2019-10-04 21:15:38,We dined here about 9 years ago and really wer...
7514,4,2017-05-29 21:34:04,3.5 stars. Came in with some friends as our fi...


(7516, 3)


#### Generate dummy data that is not available in the dataset

In [24]:
# Generate consistent values to the dataset
def generate_consistent_data(row):
    if row['rating_score'] >= 4.5:
        service = np.random.choice(['Ate there', 'Take Away', None], p=[0.4, 0.2, 0.4])
        meal_type = np.random.choice(['Luch', 'Dinner', None], p = [0.3, 0.3, 0.4])
        price_per_person_category = np.random.choice(['1-10 €', '10-20 €', '20-30 €', '30-40 €', None], p=[0.1, 0.2, 0.3, 0.1, 0.3])
        food_score = np.random.randint(4, 6)
        service_score = np.random.randint(4, 6)
        atmosphere_score = np.random.randint(4, 6)
        local_guide_reviews = np.random.randint(1, 100)
    elif row['rating_score'] >= 3.0:
        service = np.random.choice(['Ate there', 'Take Away', None], p=[0.4, 0.2, 0.4])
        meal_type = np.random.choice(['Luch', 'Dinner', None], p=[0.3, 0.3, 0.4])
        price_per_person_category = np.random.choice(['1-10 €', '10-20 €', '20-30 €', '30-40 €', None], p=[0.1, 0.2, 0.3, 0.1, 0.3])
        food_score = np.random.randint(2, 5)
        service_score = np.random.randint(2, 5)
        atmosphere_score = np.random.randint(2, 5)
        local_guide_reviews = np.random.randint(1, 100)
    else:
        service = np.random.choice(['Ate there', 'Take Away', None], p=[0.4, 0.2, 0.4])
        meal_type = np.random.choice(['Luch', 'Dinner', None], p=[0.3, 0.3, 0.4])
        price_per_person_category = np.random.choice(['1-10 €', '10-20 €', '20-30 €', '30-40 €', None], p=[0.1, 0.4, 0.3, 0.1, 0.1])
        food_score = np.random.randint(1, 4)
        service_score = np.random.randint(1, 4)
        atmosphere_score = np.random.randint(1, 4)
        local_guide_reviews = np.random.randint(1, 100)
        
    return pd.Series({
        'local_guide_reviews': local_guide_reviews,
        'service': service,
        'meal_type': meal_type,
        'price_per_person_category': price_per_person_category,
        'food_score': food_score,
        'service_score': service_score,
        'atmosphere_score': atmosphere_score,
        'recommendations_list': "['']",
    })


additional_data = reviews.apply(generate_consistent_data, axis=1)

reviews = pd.concat([reviews, additional_data], axis=1)
reviews['avg_price_per_person'] = reviews['price_per_person_category'].str.extract(r'-(\d+)\s*€')
reviews['avg_price_per_person'] = pd.to_numeric(reviews['avg_price_per_person'], errors='coerce').astype('Int64')
reviews['date'] = pd.to_datetime(reviews['date'], errors='coerce').dt.strftime('%Y-%m-%d')
display(reviews.sample(3))

,rating_score,date,review,local_guide_reviews,service,meal_type,price_per_person_category,food_score,service_score,atmosphere_score,recommendations_list,avg_price_per_person
5023,5,2019-04-05,This is a must to eat and drink at. The atmosp...,46,Ate there,None,10-20 €,5,5,4,[''],20
4447,5,2018-05-08,We had such a great time at this restaurant! S...,12,Ate there,None,20-30 €,4,5,4,[''],30
6478,5,2021-06-19,Corey was a really good waiter. Food was so go...,93,Ate there,None,1-10 €,4,4,4,[''],10


#### Save sample and summary

In [25]:
summary = reviews.groupby('rating_score').size().reset_index(name='reviews')
summary = summary.rename(columns={'rating_score': 'stars'})
summary = summary.sort_values(by='stars', ascending=False).reset_index(drop=True)
display(summary)
summary.to_csv(raw_data_path + 'resumme_' + place + '.csv', index=False)

,stars,reviews
0,5,4012
1,4,1853
2,3,788
3,2,464
4,1,399


In [ ]:
csv_file_path = '../data/processed/'
reviews.to_csv(csv_file_path + place + '_reviews.csv', index=False)
print('OK! -> processed reviews saved at', csv_file_path + place + '_reviews.csv')

OK! -> processed reviews saved at ../data/processed/Oceana Grill_reviews.csv
